In [21]:
import os
from PIL import Image
import util.image_import as ii
from glob import glob
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import matplotlib.pyplot as plt
from keras.applications import mobilenet
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, RMSprop, SGD
from util.plots import plot_history
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Subtracting mean and normalizing to save pictures

images_pot        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/potato", "*.jpg"))
images_cat        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/catfood", "*.jpg"))
images_tab        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/table", "*.jpg"))

X_train = ii.images_import(images_pot, images_cat, images_tab)

mean_image, train_pot = ii.make_images_mean(X_train, "res_still", "res_still_mean")


In [40]:
train_path = 'res_still/train'
valid_path = 'res_still/valid'
test_path = 'res_still/test'

train_path_mean = 'res_still_mean/train'
valid_path_mean = 'res_still_mean/valid'
test_path_mean = 'res_still_mean/test'

batch_size = 32

train_datagen = ImageDataGenerator(
        featurewise_center=True, 
        featurewise_std_normalization=True,
        rescale=1./255,
        rotation_range=10,
        vertical_flip=True,
        horizontal_flip=True,
        brightness_range=(0.65, 1.35))

valid_datagen = ImageDataGenerator(
        rescale=1./255
)
num_classes = 3

train_pot = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/potato", "*.jpg"))
train_cat = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/catfood", "*.jpg"))
train_tab = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/table", "*.jpg"))
val_pot   = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/valid/potato", "*.jpg"))
val_cat   = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/valid/catfood", "*.jpg"))
val_tab   = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/valid/table", "*.jpg"))
test_pot  = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/test/potato", "*.jpg"))
test_cat  = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/test/catfood", "*.jpg"))
test_tab  = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/test/table", "*.jpg"))

X_train, y_train = ii.images_to_numpy(train_pot, train_cat, train_tab)
X_val, y_val = ii.images_to_numpy(val_pot, val_cat, val_tab)
X_test, y_test = ii.images_to_numpy(test_pot, test_cat, test_tab)

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

train_datagen.fit(X_train)
valid_datagen.fit(X_train)


train_generator = train_datagen.flow(X_train,
                                     y_train,
                                     batch_size=batch_size)

train_generator = train_datagen.flow(X_val,
                                     y_val,
                                     batch_size=batch_size)


#
## Train data generator
#train_generator = train_datagen.flow_from_directory(
#        train_path,
#        target_size=(224, 224),
#        batch_size=batch_size,
#        classes=['potato', 'catfood', 'table'])
#
## Validation data generator
#validation_generator = valid_datagen.flow_from_directory(
#        valid_path,
#        target_size=(224, 224),
#        batch_size=batch_size,
#        classes=['potato', 'catfood', 'table'])
#
## this is a similar generator, for validation data
#test_generator = valid_datagen.flow_from_directory(
#        test_path,
#        target_size=(224, 224),
#        batch_size=batch_size,
#        classes=['potato', 'catfood', 'table'])

In [41]:
model = Sequential()
model.add(Flatten(input_shape=(224,224,3)))
# kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).
model.add(Dense(512 ,kernel_regularizer=regularizers.l2(0.2)))
model.add(Activation('relu'))
model.add(Dense(3, kernel_regularizer=regularizers.l2(0.2)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, epsilon=10e-8),
              metrics=['accuracy'])

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               77070848  
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 1539      
_________________________________________________________________
activation_8 (Activation)    (None, 3)                 0         
Total params: 77,072,387
Trainable params: 77,072,387
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=1000 // batch_size,
            epochs=100,
            validation_data=validation_generator,
            validation_steps=100 // batch_size)

Epoch 1/100
31/31 [==============================] - 31s 1s/step - loss: 17.7163 - acc: 0.2683 - val_loss: 10.8965 - val_acc: 0.3958
Epoch 2/100
31/31 [==============================] - 30s 968ms/step - loss: 40.4953 - acc: 0.3375 - val_loss: 70.7644 - val_acc: 0.1912
Epoch 3/100
31/31 [==============================] - 30s 959ms/step - loss: 36.4699 - acc: 0.3885 - val_loss: 18.2647 - val_acc: 0.2206
Epoch 4/100
31/31 [==============================] - 30s 955ms/step - loss: 11.3183 - acc: 0.4187 - val_loss: 4.2023 - val_acc: 0.3529
Epoch 5/100
31/31 [==============================] - 30s 957ms/step - loss: 12.3645 - acc: 0.3708 - val_loss: 35.8795 - val_acc: 0.4167
Epoch 6/100
31/31 [==============================] - 30s 981ms/step - loss: 35.5201 - acc: 0.3724 - val_loss: 19.7560 - val_acc: 0.3676
Epoch 7/100
31/31 [==============================] - 31s 1s/step - loss: 19.8522 - acc: 0.1751 - val_loss: 12.1402 - val_acc: 0.4265
Epoch 8/100
31/31 [==============================] - 32

KeyboardInterrupt: 

In [ ]:
plot_history(history)

In [6]:
imgs, labels = next(test_generator)
img = Image.fromarray(imgs[1].astype('uint8'), 'RGB')
img.show()

#ynew = model.predict(imgs)

model.evaluate(imgs,labels, batch_size=batch_size)

32/32 [==============================] - 0s 6ms/step


[58.3511962890625, 0.4375]